### Refactorización con pruebas en Python


Esta actividad contiene un ejemplo paso a paso de una sesión de refactorización guiada por pruebas. Cuando se trata de código no probado o heredado, la refactorización es peligrosa y las pruebas pueden ayudarnos a hacerlo de la manera correcta, minimizando la cantidad de errores que introducimos y posiblemente evitándolos por completo.

La refactorización no es fácil. Se requiere un doble esfuerzo para comprender el código que otros escribieron, o que escribimos en el pasado, y mover partes de él, simplificarlo, en una palabra, mejorarlo, no es algo para los pusilánimes. Al igual que la programación, la refactorización tiene sus reglas y mejores prácticas, pero puede describirse como una mezcla de técnica, intuición, experiencia y riesgo.


In [104]:
{
    "edad": 20,
    "apodo": "kapumota",
    "nombre": "Checha",
    "salario": "$943"
}

{'edad': 20, 'apodo': 'kapumota', 'nombre': 'Checha', 'salario': '$943'}

Una vez que convertimos esto a una estructura de datos de Python, obtenemos una lista de diccionarios, donde 'edad' es un número entero y los campos restantes son cadenas.

Luego, alguien escribió una clase que calcula algunas estadísticas sobre los datos de entrada. Esta clase, llamada DataStats, proporciona un solo método stats(), cuyas entradas son los datos devueltos por el servicio (en formato JSON) y dos números enteros llamados `iedad` e `isalario`. Esos, de acuerdo con la breve documentación de la clase, son la edad inicial y el salario inicial utilizados para calcular el aumento anual promedio del salario en todo el conjunto de datos.


El codigo es el siguiente:


In [105]:
%%writefile datastats.py

import math
import json


class DataStats:
  def stats(self, data, iedad, isalario):
    incremento_edad_promedio = math.floor(sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio = math.floor( sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    incremento_anual_promedio = math.floor(incremento_salario_promedio/incremento_edad_promedio)

    salarios = [int(e['salario'][1:]) for e in data]
    limite = '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return json.dumps({
        'edad_promedio': math.floor(sum([e['edad'] for e in data])/len(data)),
        'salario_promedio': math.floor(sum( [int(e['salario'][1:]) for e in data])/len(data)),
        'incremento_anual_promedio': incremento_anual_promedio,
        'max_salario': max_salario,
        'min_salario': min_salario
        })

Overwriting datastats.py


### La meta

Es bastante fácil, incluso para el ojo inexperto, detectar algunos problemas en la clase anterior. Una lista de los más llamativos es

- La clase expone un solo método y no tiene __init__(), por lo que una sola función podría proporcionar la misma funcionalidad.

- El método stats() es demasiado grande y realiza demasiadas tareas. Esto hace que la depuración sea muy difícil, ya que hay una sola pieza de código inextricable que hace todo.

 - Hay mucha duplicación de código, o al menos varias líneas que son muy similares. En particular, las dos operaciones '£' + str(max(salaries)) y './s{}'.format(str(min(salaries))), las dos líneas diferentes que comienzan con salario =, y las varias comprensiones de listas.

Entonces, dado que vamos a usar este código en alguna parte, posiblemente queremos solucionar estos problemas.

La clase, sin embargo, está funcionando perfectamente. Se ha utilizado en producción durante muchos años y no hay errores conocidos, por lo que nuestra operación debe ser una refactorización, lo que significa que queremos escribir algo mejor, conservando el comportamiento del objeto anterior.

### El camino

Aquí, quiero mostrarles cómo puede refactorizar de manera segura dicha clase usando pruebas. Esto es diferente del TDD, pero los dos están estrechamente relacionados. La clase que tenemos no se ha creado usando TDD, ya que no hay pruebas, pero podemos usar pruebas para garantizar que se conserve su comportamiento. Por lo tanto, esto debería llamarse `refactorización impulsada por pruebas` (TDR).

La idea detrás de TDR es bastante simple. Primero, tenemos que escribir una prueba que verifique el comportamiento de algún código, posiblemente una pequeña parte con un alcance y salida claramente definidos. Esta es una prueba unitaria póstuma (o tardía), y simula lo que el autor del código debería haber proporcionado.


Una vez que tengas tu prueba unitaria puedes ir y modificar el código, sabiendo que el comportamiento del objeto resultante será el mismo que el anterior. Como puede comprender fácilmente, la efectividad de esta metodología depende en gran medida de la calidad de las pruebas en sí, posiblemente más que cuando se desarrolla con TDD, y es por eso que la refactorización es difícil.

https://refactoring.guru/design-patterns/decorator


**Pregunta:** ¿Puedes encontrar algunos problemas de la clase anterior?

# Tus respuestas
Si encuentro algunos problemas de la clase anterior como:
- La clase expone un solo método stats pero no tiene un método __init__ para inicializar el objeto.
- stats realiza demasiadas tareas diferentes, como calcular estadísticas y formatear los resultados.
- La falta de pruebas unitarias por que eso dificulta su comportamiento como tambien detectar sus posibles errores

In [106]:
class DataStats:
    # Falta de encapsulación y cohesión
    # La clase no tiene un método __init__ para inicializar el objeto
    # Esto puede hacer que sea confuso para los usuarios de la clase entender cómo usarla correctamente
    def stats(self, data, iedad, isalario):
        # Método stats demasiado grande
        # Realiza demasiadas tareas diferentes, violando el principio de responsabilidad única
        # Hace que el código sea difícil de mantener y entender
        incremento_edad_promedio = math.floor(sum([e['edad'] for e in data])/len(data)) - iedad
        incremento_salario_promedio = math.floor(sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario
        incremento_anual_promedio = math.floor(incremento_salario_promedio/incremento_edad_promedio)
        salarios = [int(e['salario'][1:]) for e in data]
        limite = '$' + str(max(salarios))
        max_salario = [e for e in data if e['salario'] == limite]
        salarios = [int(d['salario'][1:]) for d in data]
        min_salario = [e for e in data if e['salario'] == '${}'.format(str(min(salarios)))]
        return json.dumps({
            'edad_promedio': math.floor(sum([e['edad'] for e in data])/len(data)),
            'salario_promedio': math.floor(sum([int(e['salario'][1:]) for e in data])/len(data)),
            'incremento_anual_promedio': incremento_anual_promedio,
            'max_salario': max_salario,
            'min_salario': min_salario
        })
    # Duplicación de código
    # Varias instancias de código duplicado, como las líneas que calculan el salario máximo y mínimo
    # Y la conversión de salario a entero

    # Falta de pruebas unitarias
    # La clase no tiene pruebas unitarias, lo que dificulta verificar su comportamiento y detectar posibles errores


#### 1 Prueba de los endpoints

Sean los siguientes datos:

In [107]:
test_data = [
    {
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
    },
    {
        "id": 2,
        "nombre": "Claudio",
        "apodo": "Avila",
        "edad": 49,
        "salario": "$67137"
    },
    {
        "id": 3,
        "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }
]

**Pregunta:** Llamando al método `stats()` con esa salida, con `iedad` establecido en `20` e `isalario` establecido en `20000`. ¿Cual es el resultado JSON?.

In [108]:
## Tus respuestas
import json

# Definimos la clase DataStats y su método stats()
class DataStats:
    def stats(self, data, iedad, isalario):
        # Realizamos Cálculos estadísticos
        incremento_edad_promedio = sum([e['edad'] for e in data]) // len(data) - iedad
        incremento_salario_promedio = sum([int(e['salario'][1:]) for e in data]) // len(data) - isalario
        incremento_anual_promedio = incremento_salario_promedio // incremento_edad_promedio

        # Encontramos el salario máximo y mínimo

        salarios = [int(e['salario'][1:]) for e in data]
        limite = '$' + str(max(salarios))
        max_salario = [e for e in data if e['salario'] == limite]
        min_salario = [e for e in data if e['salario'] == '${}'.format(str(min(salarios)))]

        # Creamos el diccionario de resultados y convertirlo a JSON
        result = {
            'edad_promedio': sum([e['edad'] for e in data]) // len(data),
            'salario_promedio': sum([int(e['salario'][1:]) for e in data]) // len(data),
            'incremento_anual_promedio': incremento_anual_promedio,
            'max_salario': max_salario,
            'min_salario': min_salario
        }
        return json.dumps(result)

# Datos de prueba
test_data = [
    {
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
    },
    {
        "id": 2,
        "nombre": "Claudio",
        "apodo": "Avila",
        "edad": 49,
        "salario": "$67137"
    },
    {
        "id": 3,
        "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }
]

# Crear una instancia de DataStats
data_stats = DataStats()

# Llamar al método stats() con los parámetros dados
resultado_json = data_stats.stats(test_data, 20, 20000)

# Imprimir el resultado JSON
print(resultado_json)


{"edad_promedio": 62, "salario_promedio": 55165, "incremento_anual_promedio": 837, "max_salario": [{"id": 3, "nombre": "Tomo", "apodo": "Frugs", "edad": 70, "salario": "$70472"}], "min_salario": [{"id": 1, "nombre": "Irene", "apodo": "Lara", "edad": 68, "salario": "$27888"}]}


In [109]:
%%writefile test_datastats.py

import json

from datastats import DataStats


def test_json():
    test_data = [
    {
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
    },
    {
        "id": 2,
        "nombre": "Claudio",
        "apodo": "Avila",
        "edad": 49,
        "salario": "$67137"
    },
    {
        "id": 3,
        "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }
]

    ds = DataStats()

    assert ds.stats(test_data, 20, 20000) == json.dumps(
        {

        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
        }]
    }

)

Overwriting test_datastats.py


In [110]:
! pytest -v test_datastats.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 1 item                                                                                   

test_datastats.py::test_json PASSED                                                          [100%]

======================================== 1 passed in 0.02s =========================================


#### 2  Deshacerse del formato JSON


El método devuelve su salida en formato JSON y mirando la clase es evidente que la conversión la realiza  `json.dumps()`.

La estructura del código es la siguiente:

In [111]:
class DataStats:
  def stats(self, data, iedad, isalario):
    [code_parte_1]

    return json.dumps({
        [code_parte_2]
        })

Donde `code_parte_2` depende de `code_parte_1`. La primera refactorización entonces sigue el siguiente procedimiento:

1. Escribimos una prueba llamada `test__stats()` para un método `_stats()` que se supone que devolverá los datos como una estructura de Python. Podemos inferir esto último manualmente desde JSON o ejecutando `json.loads()` desde un shell de Python. La prueba falla.

2. Duplicamos el código del método `stats()` que produce los datos, colocándolo en el nuevo método `_stats()`. La prueba  debe pasar.

3. Eliminamos el código duplicado en `stats()` reemplazándolo con una llamada a `_stats()`.


Ahora el código de la clase se ve así:


In [112]:
%%writefile datastats.py
import math
import json

class DataStats:

  def _stats(self, data, iedad, isalario):

    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio= math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    incremento_anual_promedio = math.floor(
            incremento_salario_promedio/incremento_edad_promedio)

    salarios = [int(e['salario'][1:]) for e in data]
    limite= '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return {
            'edad_promedio': math.floor(sum([e['edad'] for e in data])/len(data)),
            'salario_promedio': math.floor(sum(
                [int(e['salario'][1:]) for e in data])/len(data)),
            'incremento_anual_promedio': incremento_anual_promedio,
            'max_salario': max_salario,
            'min_salario': min_salario
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

Overwriting datastats.py


Y tenemos dos pruebas que comprueban la corrección de la misma.


In [113]:
%%writefile test_datastats.py

import json

from datastats import DataStats


def test_json():
    test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888"
        },

       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

    ds = DataStats()

    assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )


def test__stats():
    test_data = [
        {
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
    },
    {
        "id": 2,
        "nombre": "Claudio",
        "apodo": "Avila",
        "edad": 49,
        "salario": "$67137"
    },
    {
        "id": 3,
        "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }

    ]

    ds = DataStats()

    assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"

        }]
    }

Overwriting test_datastats.py


In [114]:
! pytest test_datastats.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 2 items                                                                                  

test_datastats.py ..                                                                         [100%]

======================================== 2 passed in 0.02s =========================================


#### 3 Refactorización de las pruebas


Es claro que la lista de diccionarios `test_data` se usará en cada prueba que realicemos, por lo que ya es hora de que la traslademos a una variable global.
También podríamos mover los datos de salida a una variable global.

El conjunto de pruebas ahora parece:

In [115]:
%%writefile test_datastats.py

import json

from datastats import DataStats


test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888"
        },

       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

def test_json():

  ds = DataStats()

  assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )


def test__stats():

   ds = DataStats()

   assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"

        }]
    }

Overwriting test_datastats.py


In [116]:
! pytest test_datastats.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 2 items                                                                                  

test_datastats.py ..                                                                         [100%]

======================================== 2 passed in 0.02s =========================================


#### 4 Aislamiento del algoritmo de edad promedio

Aislar características independientes es un objetivo clave del diseño de software. Por lo tanto, la refactorización tendrá como objetivo desentrañar el código dividiéndolo en pequeñas funciones separadas.


Para aislar algún código, lo primero que debe hacer es duplicarlo, colocándolo en un método dedicado. Como estamos refactorizando con pruebas, lo primero es escribir una prueba para este método.


In [117]:
%%writefile datastats.py
import math
import json

class DataStats:

  def _edad_promedio(self, data):
    return math.floor(sum([e['edad'] for e in data])/len(data))

  def _stats(self, data, iedad, isalario):

    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio= math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    incremento_anual_promedio = math.floor(
            incremento_salario_promedio/incremento_edad_promedio)

    salarios = [int(e['salario'][1:]) for e in data]
    limite= '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': math.floor(sum(
                [int(e['salario'][1:]) for e in data])/len(data)),
            'incremento_anual_promedio': incremento_anual_promedio,
            'max_salario': max_salario,
            'min_salario': min_salario
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

Overwriting datastats.py


In [118]:
%%writefile test_datastats.py

import json

from datastats import DataStats


test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888"
        },

       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

def test_json():

  ds = DataStats()

  assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )


def test__stats():

   ds = DataStats()

   assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"

        }]
    }

def test__edad_promedio():
  ds = DataStats()

  assert ds._edad_promedio(test_data) == 62

Overwriting test_datastats.py


In [119]:
! pytest test_datastats.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 3 items                                                                                  

test_datastats.py ...                                                                        [100%]

======================================== 3 passed in 0.02s =========================================


#### 5 Aíslamiento el algoritmo de salario promedio


In [120]:
%%writefile datastats.py
import math
import json

class DataStats:
  def _salario_promedio(self, data):
    return math.floor(sum([int(e['salario'][1:]) for e in data])/len(data))

  def _edad_promedio(self, data):
    return math.floor(sum([e['edad'] for e in data])/len(data))

  def _stats(self, data, iedad, isalario):

    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio= math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    incremento_anual_promedio = math.floor(
            incremento_salario_promedio/incremento_edad_promedio)

    salarios = [int(e['salario'][1:]) for e in data]
    limite= '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': self._salario_promedio(data),
            'incremento_anual_promedio': incremento_anual_promedio,
            'max_salario': max_salario,
            'min_salario': min_salario
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

Overwriting datastats.py


In [121]:
%%writefile test_datastats.py

import json

from datastats import DataStats


test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888"
        },

       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

def test_json():

  ds = DataStats()

  assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )


def test__stats():

   ds = DataStats()

   assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"

        }]
    }

def test__edad_promedio():
  ds = DataStats()

  assert ds._edad_promedio(test_data) == 62

def test__salario_promedio():
  ds = DataStats()
  assert ds._salario_promedio(test_data) == 55165

Overwriting test_datastats.py


In [122]:
! pytest test_datastats.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 4 items                                                                                  

test_datastats.py ....                                                                       [100%]

======================================== 4 passed in 0.03s =========================================


#### 6 Aislamiento del algoritmo de aumento anual promedio

Para el incremento promedio anual del salario tenemos una nueva prueba:

In [123]:
#def test__incremento_anual_promedio():
#  ds = DataStats()
#  assert ds._incremento_anual_promedio(test_data, 20, 20000) == 837
import math
import json

class DataStats:
    def _incremento_anual_promedio(self, data, iedad, isalario):
        incremento_edad_promedio = self._edad_promedio(data) - iedad
        incremento_salario_promedio = self._salario_promedio(data) - isalario

        return math.floor(incremento_salario_promedio / incremento_edad_promedio)

    def _salario_promedio(self, data):
        return math.floor(sum([int(e['salario'][1:]) for e in data]) / len(data))

    def _edad_promedio(self, data):
        return math.floor(sum([e['edad'] for e in data]) / len(data))

    def _stats(self, data, iedad, isalario):
        return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': self._salario_promedio(data),
            'incremento_anual_promedio': self._incremento_anual_promedio(data, iedad, isalario),
            'max_salario': self._max_salario(data),
            'min_salario': self._min_salario(data)
        }

    def stats(self, data, iedad, isalario):
        return json.dumps(self._stats(data, iedad, isalario))

    def _max_salario(self, data):
        salarios = [int(e['salario'][1:]) for e in data]
        limite = max(salarios)
        return [e for e in data if int(e['salario'][1:]) == limite]

    def _min_salario(self, data):
        salarios = [int(e['salario'][1:]) for e in data]
        limite = min(salarios)
        return [e for e in data if int(e['salario'][1:]) == limite]

# Prueba para el cálculo del incremento anual promedio del salario
def test__incremento_anual_promedio():
    test_data = [
        {"id": 1, "nombre": "Irene", "apodo": "Lara", "edad": 68, "salario": "$27888"},
        {"id": 2, "nombre": "Claudio", "apodo": "Avila", "edad": 49, "salario": "$67137"},
        {"id": 3, "nombre": "Tomo", "apodo": "Frugs", "edad": 70, "salario": "$70472"}
    ]
    ds = DataStats()
    assert ds._incremento_anual_promedio(test_data, 20, 20000) == 837


Un nuevo método que pasa la prueba:

In [124]:
#def _incremento_anual_promedio(self, data, iedad, isalario):
#  incremento_anual_promedio = math.floor(
#            sum([e['edad'] for e in data])/len(data)) - iedad
#  incremento_salario_promedio = math.floor(
#            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

#  return math.floor(incremento_salario_promedio/incremento_edad_promedio)

def _incremento_anual_promedio(self, data, iedad, isalario):
    incremento_edad_promedio = self._edad_promedio(data) - iedad
    incremento_salario_promedio = self._salario_promedio(data) - isalario

    return math.floor(incremento_salario_promedio / incremento_edad_promedio)


Y una nueva versión del método `_stats()`:

In [125]:
#def _stats(self, data, iedad, isalario):
#  salarios = [int(e['salario'][1:]) for e in data]
#  limite= '$' + str(max(salarios))
#
#  max_salario = [e for e in data if e['salario'] == limite]
#
#  salarios = [int(d['salario'][1:]) for d in data]
#  min_salario = [e for e in data if e['salario'] ==
#                      '${}'.format(str(min(salarios)))]
#
#  return {
#        'edad_promedio': self._avg_age(data),
#        'salario_promedio':  self._salario_promedio(data),
#        'incremento_anual_promedio': self._incremento_anual_promedio (
#                data, iedad, isalario),
#        'max_salario': max_salario,
#        'min_salario': min_salario
#        }

def _stats(self, data, iedad, isalario):
    return {
        'edad_promedio': self._edad_promedio(data),
        'salario_promedio': self._salario_promedio(data),
        'incremento_anual_promedio': self._incremento_anual_promedio(data, iedad, isalario),
        'max_salario': self._max_salario(data),
        'min_salario': self._min_salario(data)
    }

def _max_salario(self, data):
    salarios = [int(e['salario'][1:]) for e in data]
    max_salary = max(salarios)
    return [e for e in data if int(e['salario'][1:]) == max_salary]

def _min_salario(self, data):
    salarios = [int(e['salario'][1:]) for e in data]
    min_salary = min(salarios)
    return [e for e in data if int(e['salario'][1:]) == min_salary]


Con todo esto tenemos:

In [126]:
%%writefile datastats.py
import math
import json

class DataStats:

  def _salario_promedio(self, data):
    return math.floor(sum([int(e['salario'][1:]) for e in data])/len(data))

  def _edad_promedio(self, data):
    return math.floor(sum([e['edad'] for e in data])/len(data))

  def _incremento_anual_promedio(self, data, iedad, isalario):

    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio = math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    return math.floor(incremento_salario_promedio/incremento_edad_promedio)

  def _stats(self, data, iedad, isalario):
    salarios = [int(e['salario'][1:]) for e in data]
    limite= '$' + str(max(salarios))

    max_salario = [e for e in data if e['salario'] == limite]

    salarios = [int(d['salario'][1:]) for d in data]
    min_salario = [e for e in data if e['salario'] ==
                      '${}'.format(str(min(salarios)))]

    return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': self._salario_promedio(data),
            'incremento_anual_promedio': self._incremento_anual_promedio(
                data, iedad, isalario),
            'max_salario': max_salario,
            'min_salario': min_salario
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

Overwriting datastats.py


In [127]:
%%writefile test_datastats.py

import json

from datastats import DataStats


test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888"
        },

       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

def test_json():

  ds = DataStats()

  assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )


def test__stats():

   ds = DataStats()

   assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"

        }]
    }

def test__edad_promedio():
  ds = DataStats()

  assert ds._edad_promedio(test_data) == 62

def test__salario_promedio():
  ds = DataStats()
  assert ds._salario_promedio(test_data) == 55165

def test__incremento_anual_promedio():
  ds = DataStats()

  assert ds._incremento_anual_promedio(test_data, 20, 20000) == 837


Overwriting test_datastats.py


In [128]:
! pytest test_datastats.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 5 items                                                                                  

test_datastats.py .....                                                                      [100%]

======================================== 5 passed in 0.02s =========================================


### 7 - Aislamiento los algoritmos de salario máximo y mínimo


Para este caso identificamos las nuevas pruebas:

In [129]:
%%writefile datastats.py
import math
import json

class DataStats:

  def _salario_promedio(self, data):
    return math.floor(sum([int(e['salario'][1:]) for e in data])/len(data))

  def _edad_promedio(self, data):
    return math.floor(sum([e['edad'] for e in data])/len(data))

  def _incremento_anual_promedio(self, data, iedad, isalario):

    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio = math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    return math.floor(incremento_salario_promedio/incremento_edad_promedio)

  def _max_salario(self, data):
    salarios = [int(e['salario'][1:]) for e in data]
    limite = '$' + str(max(salarios))

    return [e for e in data if e['salario'] == limite]

  def _min_salario(self, data):
    salarios = [int(d['salario'][1:]) for d in data]
    return [e for e in data if e['salario'] ==
                '${}'.format(str(min(salarios)))]


  def _stats(self, data, iedad, isalario):
    return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': self._salario_promedio(data),
            'incremento_anual_promedio': self._incremento_anual_promedio(
                data, iedad, isalario),
            'max_salario': self._max_salario(data),
            'min_salario': self._min_salario(data)
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

Overwriting datastats.py


In [130]:
%%writefile test_datastats.py

import json

from datastats import DataStats

test_data = [
        {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888"
        },

       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }
    ]

def test_json():

  ds = DataStats()

  assert ds.stats(test_data, 20, 20000) == json.dumps(
        {
            "edad_promedio": 62,
            "salario_promedio": 55165,
            "incremento_anual_promedio": 837,
            "max_salario": [{
                 "id": 3,
                  "nombre": "Tomo",
                  "apodo": "Frugs",
                  "edad": 70,
                  "salario": "$70472"
           }],
          "min_salario": [{
              "id": 1,
              "nombre": "Irene",
              "apodo": "Lara",
              "edad": 68,
              "salario": "$27888"
            }]
       }
    )


def test__stats():

   ds = DataStats()

   assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
             "edad": 70,
              "salario": "$70472"
    }],
    "min_salario": [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"

        }]
    }

def test__edad_promedio():
  ds = DataStats()
  assert ds._edad_promedio(test_data) == 62

def test__salario_promedio():
  ds = DataStats()
  assert ds._salario_promedio(test_data) == 55165

def test__incremento_anual_promedio():
  ds = DataStats()
  assert ds._incremento_anual_promedio(test_data, 20, 20000) == 837


def test__max_salario():
  ds = DataStats()

  assert ds._max_salario(test_data) == [{
        "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
    }]

def test__min_salario():
  ds = DataStats()
  assert ds._min_salario(test_data) ==[{
      "id": 1,
      "nombre": "Irene",
      "apodo": "Lara",
      "edad": 68,
      "salario": "$27888"
       }]

Overwriting test_datastats.py


In [131]:
! pytest -v test_datastats.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 7 items                                                                                  

test_datastats.py::test_json PASSED                                                          [ 14%]
test_datastats.py::test__stats PASSED                                                        [ 28%]
test_datastats.py::test__edad_promedio PASSED                                                [ 42%]
test_datastats.py::test__salario_promedio PASSED                                             [ 57%]
test_datastats.py::test__incremento_anual_promedio PASSED                                    [ 71%]
test_datastats.py::test__max_salario PASSED                                                  [ 85%]
test_datastats.py::test__min_salario PASSED

In [132]:
! pip install coverage
! pip install pytest-cov

In [133]:
!pytest --cov=datastats datastats.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 0 items                                                                                  


---------- coverage: platform linux, python 3.10.12-final-0 ----------
Name           Stmts   Miss  Cover
----------------------------------
datastats.py      22     12    45%
----------------------------------
TOTAL             22     12    45%

====================================== no tests ran in 0.04s =======================================


#### 8 Reducción de la duplicación de código

Del código anterior los dos métodos `_max_salario()` y `_min_salario()` comparten una gran cantidad de código, aunque el segundo es más conciso.


In [134]:
def _max_salario(self, data):
  salarios = [int(e['salario'][1:]) for e in data]
  limite = '$' + str(max(salarios))

  return [e for e in data if e['salario'] == limite]

def _min_salario(self, data):
  salarios = [int(d['salario'][1:]) for d in data]
  return [e for e in data if e['salario'] ==
                '$'.format(str(min(salarios)))]

Veamos el caso de la variable `limite` en la segunda función. Tan pronto como se cambie algo, ejecuta las pruebas para verificar que el comportamiento externo no haya cambiado.

In [135]:
def _max_salario(self, data):
  salarios = [int(e['salario'][1:]) for e in data]
  limite = '$' + str(max(salarios))

  return [e for e in data if e['salario'] == limite]

def _min_salario(self, data):
  salarios = [int(d['salario'][1:]) for d in data]
  limite = '$'.format(str(min(salarios)))

  return [e for e in data if e['salario'] == limite]

Ahora las dos funciones son las mismas pero para las funciones `min()` y `max()`. Todavía usan diferentes nombres de variables y diferentes códigos para formatear el limite, por lo que una primera acción es igualarlos, copiando el código de `_min_salario()` a `_max_salario()` y cambiando `min()` a `max()`-


In [136]:
def _max_salario(self, data):
  salarios = [int(e['salario'][1:]) for d in data]
  limite = '$'.format(str(max(salarios)))

  return [e for e in data if e['salario'] == limite]

def _min_salario(self, data):
  salarios = [int(d['salario'][1:]) for d in data]
  limite = '$'.format(str(min(salarios)))

  return [e for e in data if e['salario'] == limite]

Creamos la función `_select_salario()` que duplique ese código y acepte una función en lugar de `min()` o `max()`.

In [137]:
def _select_salario(self, data, func):
  salarios = [int(d['salario'][1:]) for d in data]
  limite = '${}'.format(str(func(salarios)))

  return [e for e in data if e['salario'] == limite]

def _max_salario(self, data):
  return self._select_salario(data, max)

def _min_salario(self, data):
  return self._select_salario(data, min)

Hay una duplicación de código entre `_salario_promedio()` y `_select_salario()`.


In [138]:
def _salario_promedio(self, data):
  return math.floor(sum([int(e['salario'][1:]) for e in data])/len(data))

In [140]:
def _select_salario(self, data, func):
  salarios = [int(d['salario'][1:]) for d in data]

Extraemos el algoritmo común en un método llamado `_salarios()`, para ello primero escribimos la prueba.

In [141]:
def test_salarios():
  ds = DataStats()
  assert ds._salarios(test_data) == [27888, 67137, 70472]

Implementamos el método.

In [142]:
def _salarios(self, data):
  return [int(d['salario'][1:]) for d in data]

Reemplazamos el código duplicado con una llamada al nuevo método.

In [143]:
def _salarios(self, data):
  return [int(d['salario'][1:]) for d in data]

In [144]:
def _select_salario(self, data, func):
  limite= '${}'.format(str(func(self._salarios(data))))
  return [e for e in data if e['salario'] == limite]

Veamos el código de `_incremento_anual_promedio()`. ¿Cuál es el problema del código?

In [145]:
def _incremento_anual_promedio(self, data, iedad, isalario):
  incremento_edad_promedio = math.floor(
      sum([e['edad'] for e in data])/len(data)) - iedad

  incremento_salario_promedio = math.floor(
      sum(self._salarios(data))/len(data)) - isalario

  return math.floor(incremento_salario_promedio/incremento_edad_promedio)

El código de `_incremento_anual_promedio()` contiene el código de los métodos `_edad_promedio()` y `_salario_promedio()`, por lo que vale la pena reemplazarlo con dos llamadas.

In [146]:
def _incremento_anual_promedio(self, data, iedad, isalario):
  incremento_edad_promedio= self._edad_promedio(data) - iedad
  incremento_salario_promedio = self._salario_promedio(data) - isalario

  return math.floor(incremento_salario_promedio/incremento_edad_promedio)

**Pregunta:** Edita los archivos necesarios para ejecutar la prueba. ¿Cuál es el código de cobertura para este ejemplo?.

In [147]:
# datastats.py
import math
import json

class DataStats:

  def _salario_promedio(self, data):
    return math.floor(sum(self._salarios(data))/len(data))

  def _edad_promedio(self, data):
    return math.floor(sum([e['edad'] for e in data])/len(data))

  def _incremento_anual_promedio(self, data, iedad, isalario):
    incremento_edad_promedio = self._edad_promedio(data) - iedad
    incremento_salario_promedio = self._salario_promedio(data) - isalario
    return math.floor(incremento_salario_promedio/incremento_edad_promedio)

  def _max_salario(self, data):
    return self._select_salario(data, max)

  def _min_salario(self, data):
    return self._select_salario(data, min)

  def _stats(self, data, iedad, isalario):
    return {
        'edad_promedio': self._edad_promedio(data),
        'salario_promedio': self._salario_promedio(data),
        'incremento_anual_promedio': self._incremento_anual_promedio(data, iedad, isalario),
        'max_salario': self._max_salario(data),
        'min_salario': self._min_salario(data)
    }

  def stats(self, data, iedad, isalario):
    return json.dumps(self._stats(data, iedad, isalario))

  def _salarios(self, data):
    return [int(d['salario'][1:]) for d in data]

  def _select_salario(self, data, func):
    limite = '${}'.format(str(func(self._salarios(data))))
    return [e for e in data if e['salario'] == limite]

# test_datastats.py
import json
from datastats import DataStats

test_data = [
    {
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
    },
    {
        "id": 2,
        "nombre": "Claudio",
        "apodo": "Avila",
        "edad": 49,
        "salario": "$67137"
    },
    {
        "id": 3,
        "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }
]

def test_json():
    ds = DataStats()
    assert ds.stats(test_data, 20, 20000) == json.dumps({
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
            "edad": 70,
            "salario": "$70472"
        }],
        "min_salario": [{
            "id": 1,
            "nombre": "Irene",
            "apodo": "Lara",
            "edad": 68,
            "salario": "$27888"
        }]
    })

def test__stats():
    ds = DataStats()
    assert ds._stats(test_data, 20, 20000) == {
        "edad_promedio": 62,
        "salario_promedio": 55165,
        "incremento_anual_promedio": 837,
        "max_salario": [{
            "id": 3,
            "nombre": "Tomo",
            "apodo": "Frugs",
            "edad": 70,
            "salario": "$70472"
        }],
        "min_salario": [{
            "id": 1,
            "nombre": "Irene",
            "apodo": "Lara",
            "edad": 68,
            "salario": "$27888"
        }]
    }

def test__edad_promedio():
    ds = DataStats()
    assert ds._edad_promedio(test_data) == 62

def test__salario_promedio():
    ds = DataStats()
    assert ds._salario_promedio(test_data) == 55165

def test__incremento_anual_promedio():
    ds = DataStats()
    assert ds._incremento_anual_promedio(test_data, 20, 20000) == 837

def test__max_salario():
    ds = DataStats()
    assert ds._max_salario(test_data) == [{
        "id": 3,
        "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }]

def test__min_salario():
    ds = DataStats()
    assert ds._min_salario(test_data) == [{
        "id": 1,
        "nombre": "Irene",
        "apodo": "Lara",
        "edad": 68,
        "salario": "$27888"
    }]

def test__salarios():
    ds = DataStats()
    assert ds._salarios(test_data) == [27888, 67137, 70472]

def test__select_salario():
    ds = DataStats()
    assert ds._select_salario(test_data, max) == [{
        "id": 3,
        "nombre": "Tomo",
        "apodo": "Frugs",
        "edad": 70,
        "salario": "$70472"
    }]

# Este es el comando para ejecutar la prueba y obtener el código de cobertura
# pytest --cov=datastats test_datastats.py


In [148]:
!pytest --cov=datastats test_datastats.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 7 items                                                                                  

test_datastats.py .......                                                                    [100%]

---------- coverage: platform linux, python 3.10.12-final-0 ----------
Name           Stmts   Miss  Cover
----------------------------------
datastats.py      22      0   100%
----------------------------------
TOTAL             22      0   100%


======================================== 7 passed in 0.04s =========================================


#### 9 Refactorización avanzada

El objetivo es cambiar la clase actual para que coincida con la nueva API y luego crear una clase que envuelva la primera y proporcione la API anterior. La estrategia no es tan diferente de lo que hicimos anteriormente, solo que esta vez trataremos con clases en lugar de métodos. Con un estupendo esfuerzo de mi imaginación nombré a la nueva clase `NDataStats`.

Lo primero, como sucede muy a menudo con la refactorización es duplicar el código y cuando insertamos código nuevo necesitamos tener pruebas que lo justifiquen.

Las pruebas serán las mismas que antes, ya que la nueva clase proporcionará las mismas funcionalidades que la anterior, así que solo crea un nuevo archivo, llamado `test_ndatastats.py` y allí se coloca la primera prueba `test_init()`.

In [149]:
%%writefile test_ndatastats.py
import json

from datastats import NDataStats
test_data = [
    {
          "id": 1,
          "nombre": "Irene",
          "apodo": "Lara",
          "edad": 68,
          "salario": "$27888"
        },

       {
           "id": 2,
          "nombre": "Claudio",
          "apodo": "Avila",
          "edad": 49,
          "salario": "$67137"
      },
      {
          "id": 3,
          "nombre": "Tomo",
          "apodo": "Frugs",
          "edad": 70,
          "salario": "$70472"
      }

]

def test_init():
  ds = NDataStats(test_data)
  assert ds.data == test_data

Overwriting test_ndatastats.py


**Pregunta:** Escribe la nueva clase `NdataStats` en el archivo `datastats.py` y comprueba si la prueba anterior pasa.

In [150]:
# Tu respuesta.
import math
import json

class DataStats:

  def _salario_promedio(self, data):
    return math.floor(sum([int(e['salario'][1:]) for e in data])/len(data))

  def _edad_promedio(self, data):
    return math.floor(sum([e['edad'] for e in data])/len(data))

  def _incremento_anual_promedio(self, data, iedad, isalario):

    incremento_edad_promedio = math.floor(
            sum([e['edad'] for e in data])/len(data)) - iedad
    incremento_salario_promedio = math.floor(
            sum([int(e['salario'][1:]) for e in data])/len(data)) - isalario

    return math.floor(incremento_salario_promedio/incremento_edad_promedio)

  def _max_salario(self, data):
    salarios = [int(e['salario'][1:]) for e in data]
    limite = '$' + str(max(salarios))

    return [e for e in data if e['salario'] == limite]

  def _min_salario(self, data):
    salarios = [int(d['salario'][1:]) for d in data]
    return [e for e in data if e['salario'] ==
                '${}'.format(str(min(salarios)))]


  def _stats(self, data, iedad, isalario):
    return {
            'edad_promedio': self._edad_promedio(data),
            'salario_promedio': self._salario_promedio(data),
            'incremento_anual_promedio': self._incremento_anual_promedio(
                data, iedad, isalario),
            'max_salario': self._max_salario(data),
            'min_salario': self._min_salario(data)
        }

  def stats(self, data, iedad, isalario):
    return json.dumps(
      self._stats(data, iedad, isalario)
  )

class NDataStats:
  def __init__(self, data):
    self.data = data



Ahora iniciamos un proceso iterativo:

- Copia una de las pruebas de `DataStats` y adapta  a `NDataStats`
- Copia algo de código de `DataStats` a `NDataStats`, adaptándolo a la nueva API y haciendo pasar una prueba.

En este punto, eliminar iterativamente métodos de `DataStats` y reemplazarlos con una llamada a `NDataStats` es una exageración.

Un ejemplo de las pruebas resultantes para `NDataStats` es el siguiente:

In [151]:
#def test_edad():
#  ds = NDataStats(test_data)
#   assert ds._edades() == [68, 49, 70]




Y el código que pasa la prueba para este caso es:

In [152]:
#def _edades(self):
#  return [d['edad'] for d in self.data]


Los métodos como `_edades()` ya no requieren un parámetro de entrada y se pueden convertir convertirlos en propiedades, cambiando las pruebas con `@property`.

Es hora de reemplazar los métodos de `DataStats` con llamadas a `NDataStats`. Podríamos hacerlo método por método, pero en realidad lo único que realmente necesitamos es reemplazar `stats()`. Así que el nuevo código es:

In [153]:
#class DataStats:
#  def stats(self, data, iedad, isalario):
#    nds = NDataStats(data)
#    return nds.stats(iedad, isalario)

**Pregunta:** Edita los archivos necesarios para ejecutar la prueba. ¿Cuál es el código de cobertura para este ejemplo ?.

In [154]:
# Tu respuesta



In [155]:
# datastats.py

import math
import json

class NDataStats:
    def __init__(self, data):
        self.data = data

    def _salario_promedio(self):
        return math.floor(sum([int(e['salario'][1:]) for e in self.data])/len(self.data))

    def _edad_promedio(self):
        return math.floor(sum([e['edad'] for e in self.data])/len(self.data))

    def _incremento_anual_promedio(self, iedad, isalario):
        incremento_edad_promedio = self._edad_promedio() - iedad
        incremento_salario_promedio = self._salario_promedio() - isalario

        return math.floor(incremento_salario_promedio/incremento_edad_promedio)

    def _max_salario(self):
        salarios = [int(e['salario'][1:]) for e in self.data]
        limite = '$' + str(max(salarios))

        return [e for e in self.data if e['salario'] == limite]

    def _min_salario(self):
        salarios = [int(d['salario'][1:]) for d in self.data]
        return [e for e in self.data if e['salario'] == '${}'.format(str(min(salarios)))]

    def _stats(self, iedad, isalario):
        return {
            'edad_promedio': self._edad_promedio(),
            'salario_promedio': self._salario_promedio(),
            'incremento_anual_promedio': self._incremento_anual_promedio(iedad, isalario),
            'max_salario': self._max_salario(),
            'min_salario': self._min_salario()
        }

    def stats(self, iedad, isalario):
        return json.dumps(self._stats(iedad, isalario))


# datastats.py - adaptación para usar NDataStats

class DataStats:
    def stats(self, data, iedad, isalario):
        nds = NDataStats(data)
        return nds.stats(iedad, isalario)

In [156]:
!pytest --cov=datastats test_datastats.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 7 items                                                                                  

test_datastats.py .......                                                                    [100%]

---------- coverage: platform linux, python 3.10.12-final-0 ----------
Name           Stmts   Miss  Cover
----------------------------------
datastats.py      22      0   100%
----------------------------------
TOTAL             22      0   100%


======================================== 7 passed in 0.04s =========================================


**Pregunta:** La refactorización es un proceso iterativo, a menudo sucederá que crees que hiciste todo lo posible, solo para descubrir más tarde que te perdiste algo. En este caso, el paso faltante es una pequeña duplicación de código.

Las dos funciones comparten la misma lógica, por lo que definitivamente podemos aislar esto  y llamar al código común en cada función.

In [157]:
#def _salario_promedio(self):
#  return math.floor(sum(self._salarios)/len(self.data))

#def _edad_promedio(self):
#  return math.floor(sum(self._edades)/len(self.data))

In [158]:
#def _promedio_floor(self, suma_de_numberos):
#  return math.floor(suma_de_numeros / len(self.data))

#def _salario_promedio(self):
#  return self._promedio_floor(sum(self._salarios))

#def _edad_promedio(self):
#  return self._promedio_floor(sum(self._edades))

Edita los archivos necesarios para ejecutar la prueba. ¿Cuál es el código de cobertura para este ejemplo.?

In [159]:
## Tu respuesta.
class NDataStats:
    def __init__(self, data):
        self.data = data
        self._salarios = [int(d['salario'][1:]) for d in data]
        self._edades = [d['edad'] for d in data]

    @property
    def _salario_promedio(self):
        return self._promedio_floor(sum(self._salarios))

    @property
    def _edad_promedio(self):
        return self._promedio_floor(sum(self._edades))

    def _promedio_floor(self, suma_de_numeros):
        return math.floor(suma_de_numeros / len(self.data))

    def _select_salario(self, func):
        salarios = [int(d['salario'][1:]) for d in self.data]
        limite = '${}'.format(str(func(salarios)))
        return [e for e in self.data if e['salario'] == limite]

    def _max_salario(self):
        return self._select_salario(max)

    def _min_salario(self):
        return self._select_salario(min)

    def stats(self, iedad, isalario):
        return {
            'edad_promedio': self._edad_promedio,
            'salario_promedio': self._salario_promedio,
            'incremento_anual_promedio': self._incremento_anual_promedio(iedad, isalario),
            'max_salario': self._max_salario(),
            'min_salario': self._min_salario()
        }


In [160]:
!pytest --cov=datastats test_datastats.py


======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: cov-5.0.0, anyio-3.7.1
collected 7 items                                                                                  

test_datastats.py .......                                                                    [100%]

---------- coverage: platform linux, python 3.10.12-final-0 ----------
Name           Stmts   Miss  Cover
----------------------------------
datastats.py      22      0   100%
----------------------------------
TOTAL             22      0   100%


======================================== 7 passed in 0.04s =========================================


**Referencias**

- [Why do developers hate code coverage? And why they should not hate it!](https://www.effective-software-testing.com/why-do-developers-hate-code-coverage)
- [Do unit tests make refactoring harder?](https://www.effective-software-testing.com/do-unit-tests-make-refactoring-harder).